In [15]:
import os

# Define the target directory
target_directory = (
    r"C:\Users\pablosal\Desktop\gbbai-azure-aoai"  # change your directory here
)

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory C:\Users\pablosal\Desktop\gbbai-azure-aoai does not exist.


In [16]:
import instructor
from openai import AzureOpenAI
import os

client = instructor.from_openai(AzureOpenAI(azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
                                            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
                                            api_key=os.getenv("AZURE_OPENAI_KEY")))


## Build System Prompt

The `SYSTEM_PROMPT` is designed to ensure that the assistant provides accurate and clear answers about events using only the provided `REFERENCE_TEXT`. Here are the main considerations:

1. **Use REFERENCE_TEXT Only**:
   - Answer questions strictly using the provided `REFERENCE_TEXT`.
   - Prefer the most recent relevant piece of `REFERENCE_TEXT` if items have a Published Date.

2. **Short Summary**:
   - Provide concise answers without mentioning the `REFERENCE_TEXT`.

3. **Irrelevant Questions**:
   - If the question isn't about the provided events, the `REFERENCE_TEXT` is empty, or the answer can't be derived, respond that you are unable to answer.
   - Set the `answered` parameter to `false` if the question can't be answered; otherwise, set it to `true`.

4. **No External Suggestions**:
   - Do not suggest checking other websites or sources.

5. **Time-Related Responses**:
   - Convert and return both date and time in the user's time zone.
   - Handle daylight saving time changes appropriately.
   - Default to UTC if the user's time zone or region is not provided.
   - Ensure a consistent and clear date and time format (e.g., `YYYY-MM-DD HH:MM:SS`).
   - Adjust the date for events that cross into the next day in UTC to reflect the correct local date and time.
   - Validate the converted time to account for any potential discrepancies in time zone data.
   - Include the local time zone, the current time, and whether daylight saving time is in effect in the response.

6. **Response Format**:
   - Send the answer in unformatted JSON format.

In [17]:
SYSTEM_PROMPT = (
    "You are an assistant whose job is to provide accurate answers about events using only the provided REFERENCE_TEXT. "
    "Follow these guidelines to ensure clarity and precision in your responses:\n\n"
    "1. **Use REFERENCE_TEXT Only**: Answer questions using only the REFERENCE_TEXT. If the items have a Published Date, use the most recent relevant piece of REFERENCE_TEXT rather than the last item.\n"
    "2. **Short Summary**: Provide answers in the form of a short summary without mentioning that the answer came from the REFERENCE_TEXT.\n"
    "3. **Irrelevant Questions**: If the question isn't about the provided events, the REFERENCE_TEXT is empty, or you can't answer, disregard the question and reply that you are unable to answer it.\n"
    "4. **Answered Parameter**: If you can't answer the question, set the answered parameter to false; otherwise, set it to true.\n"
    "5. **No External Suggestions**: Do not suggest checking other websites or other reputable sources.\n\n"
    "6. **Structured Response for Events**: If events are mentioned in the REFERENCE_TEXT, ensure the response includes the ID, date, and name of each event.\n\n"    
    "When sending back a time-related response, ensure both the date and time are converted and returned in the user's time zone. Follow these additional guidelines:\n\n"
    "1. **Daylight Saving Time**: Handle daylight saving time changes appropriately.\n"
    "2. **Default to UTC**: If the user's time zone or region is not provided, default to UTC.\n"
    "3. **Consistent Format**: Ensure the date and time format is consistent and clear (e.g., YYYY-MM-DD HH:MM:SS).\n"
    "4. **Cross-Day Events**: If the event occurs at a time that crosses into the next day in UTC, adjust the date accordingly to reflect the correct local date and time.\n"
    "5. **Validate Time**: Validate the converted time to account for any potential discrepancies in time zone data.\n"
    "6. **Include Time Zone Info**: Always include the local time zone, the current time, and whether daylight saving time is in effect in the response.\n\n"
    "Send the answer in unformatted JSON format."
)

## Build Golden Dataset

The code provided is designed to create a "golden dataset" of events to test the system's ability to handle and correctly output data across different time zones. Here's a step-by-step explanation of the code:

1. **Import Libraries**:
   - `pandas` for data manipulation.
   - `pytz` for timezone handling.
   - `datetime` and `timedelta` for date and time operations.
   - `random` for generating random events.
   - `typing` for type annotations.

2. **Define Regions and Time Zones**:
   - A dictionary `regions` maps region names to their respective time zones.

3. **Generate Events**:
   - `generate_events` function creates a list of random events within a specified date range.
   - Each event has an `eventId`, `eventDate`, and `eventName`.

4. **Sort Events by Date**:
   - `sort_events_by_date` function sorts the list of events by their date.

5. **Convert Event Time**:
   - `convert_event_time` function converts an event's date to a specified time zone.

6. **Find Closest Event**:
   - `find_closest_event` function finds the event closest to the current date.

7. **Create Event Dataset**:
   - `create_event_dataset` function creates a DataFrame with the closest event and its time translated across different regions.
   - It generates 10 different sets of 10 events, finds the closest event in each set, and converts its time to different time zones.

8. **Main Execution**:
   - Define the start and end dates for event generation.
   - Generate 100 random events within the specified date range.
   - Get the current date in UTC.
   - Create the event dataset and display the first few rows.

In [18]:
import pandas as pd
import pytz
from datetime import datetime, timedelta
import random
from typing import List, Dict, Any

# Define regions and time zones
regions = {
    "Los_Angeles": "America/Los_Angeles",
    "Chicago": "America/Chicago",
    "New_York": "America/New_York",
    "London": "Europe/London",
    "UTC": "UTC"
}

def generate_events(num_events: int, start_date: datetime, end_date: datetime) -> List[Dict[str, Any]]:
    """
    Generate a list of random events within a specified date range.

    Parameters:
    - num_events (int): Number of events to generate.
    - start_date (datetime): Start date of the range.
    - end_date (datetime): End date of the range.

    Returns:
    - List[Dict[str, Any]]: List of events with eventId, eventDate, and eventName.
    """
    event_list = []
    for i in range(num_events):
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days),
                                             seconds=random.randint(0, 86400))
        event_list.append({
            "eventId": 1000 + i,
            "eventDate": random_date.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "eventName": f"Event {i + 1}"
        })
    return event_list

def sort_events_by_date(events: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Sort events by date.

    Parameters:
    - events (List[Dict[str, Any]]): List of events.

    Returns:
    - List[Dict[str, Any]]: Sorted list of events.
    """
    return sorted(events, key=lambda x: datetime.strptime(x["eventDate"], "%Y-%m-%dT%H:%M:%SZ"))

def convert_event_time(event_date_str: str, target_tz_str: str) -> str:
    """
    Convert event date to different time zones.

    Parameters:
    - event_date_str (str): Event date in string format.
    - target_tz_str (str): Target time zone string.

    Returns:
    - str: Event date converted to the target time zone.
    """
    event_date_utc = datetime.strptime(event_date_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.utc)
    target_tz = pytz.timezone(target_tz_str)
    return event_date_utc.astimezone(target_tz).strftime("%Y-%m-%d %H:%M:%S %Z")

def find_closest_event(events: List[Dict[str, Any]], current_date: datetime) -> Dict[str, Any]:
    """
    Find the first event that is closest to the current date.

    Parameters:
    - events (List[Dict[str, Any]]): List of events.
    - current_date (datetime): The current date.

    Returns:
    - Dict[str, Any]: The closest event.
    """
    sorted_events = sort_events_by_date(events)
    closest_event = min(sorted_events, key=lambda x: abs(pytz.utc.localize(datetime.strptime(x["eventDate"], "%Y-%m-%dT%H:%M:%SZ")) - current_date))
    return closest_event

def create_event_dataset(events: List[Dict[str, Any]], regions: Dict[str, str], current_date: datetime) -> pd.DataFrame:
    """
    Create a DataFrame with the closest event and its time translated across regions.

    Parameters:
    - events (List[Dict[str, Any]]): List of events.
    - regions (Dict[str, str]): Dictionary of regions and their time zones.
    - current_date (datetime): Current date.

    Returns:
    - pd.DataFrame: DataFrame with the closest event and its translated times.
    """
    data = []

    for _ in range(10):  # Generate 10 different sets of 10 events
        event_subset = random.sample(events, 10)
        closest_event = find_closest_event(event_subset, current_date)
        row = {"Events": event_subset,
               "Closest_Event": closest_event}

        for region, tz in regions.items():
            row[region] = convert_event_time(closest_event["eventDate"], tz)

        data.append(row)

    return pd.DataFrame(data)

start_date = datetime(2024, 8, 1)
end_date = datetime(2024, 9, 1)

events = generate_events(100, start_date, end_date)
current_date = datetime.now(pytz.utc)

event_dataset = create_event_dataset(events, regions, current_date)
event_dataset.head()


,Events,Closest_Event,Los_Angeles,Chicago,New_York,London,UTC
0,"[{'eventId': 1091, 'eventDate': '2024-08-16T19...","{'eventId': 1001, 'eventDate': '2024-08-09T00:...",2024-08-08 17:33:22 PDT,2024-08-08 19:33:22 CDT,2024-08-08 20:33:22 EDT,2024-08-09 01:33:22 BST,2024-08-09 00:33:22 UTC
1,"[{'eventId': 1003, 'eventDate': '2024-08-09T23...","{'eventId': 1083, 'eventDate': '2024-08-04T12:...",2024-08-04 05:34:51 PDT,2024-08-04 07:34:51 CDT,2024-08-04 08:34:51 EDT,2024-08-04 13:34:51 BST,2024-08-04 12:34:51 UTC
2,"[{'eventId': 1031, 'eventDate': '2024-08-31T07...","{'eventId': 1063, 'eventDate': '2024-08-04T08:...",2024-08-04 01:09:39 PDT,2024-08-04 03:09:39 CDT,2024-08-04 04:09:39 EDT,2024-08-04 09:09:39 BST,2024-08-04 08:09:39 UTC
3,"[{'eventId': 1021, 'eventDate': '2024-08-18T11...","{'eventId': 1047, 'eventDate': '2024-08-07T02:...",2024-08-06 19:14:33 PDT,2024-08-06 21:14:33 CDT,2024-08-06 22:14:33 EDT,2024-08-07 03:14:33 BST,2024-08-07 02:14:33 UTC
4,"[{'eventId': 1030, 'eventDate': '2024-08-21T01...","{'eventId': 1088, 'eventDate': '2024-08-14T15:...",2024-08-14 08:19:40 PDT,2024-08-14 10:19:40 CDT,2024-08-14 11:19:40 EDT,2024-08-14 16:19:40 BST,2024-08-14 15:19:40 UTC


## Preprocess Prompt

In this section, we will preprocess the prompt by adding the current date and local time. This ensures that the prompt contains relevant temporal information, which can be crucial for handling time-based queries accurately.

#### 🕒 Get User Time Information Function:

- **`get_user_time_info`**: This function retrieves the current time and daylight saving status for a given user timezone.
  - **Parameters**:
    - `user_timezone_str` (str): The user's timezone as a string.
  - **Returns**:
    - `Dict[str, Any]`: A dictionary containing the formatted time information and daylight saving status.
  - **Steps**:
    1. **Get User's Time Zone**: Uses `pytz.timezone` to get the user's time zone.
    2. **Get Current Time**: Uses `datetime.now` with the user's time zone to get the current time.
    3. **Check Daylight Saving**: Checks if daylight saving time is in effect using `current_time.dst()`.
    4. **Get Time Zone Abbreviation**: Gets the time zone abbreviation using `strftime('%Z')`.
    5. **Format Response**: Creates a dictionary with the time information and daylight saving status.
    6. **Error Handling**: Returns an error message if an exception occurs.

#### 📝 Handle User Query Function:

- **`handle_user_query`**: This function handles the user's query by appending the current time information.
  - **Parameters**:
    - `user_query` (str): The user's query.
    - `user_timezone_str` (str): The user's timezone as a string.
  - **Returns**:
    - `str`: The full response with the time information and user's query.
  - **Steps**:
    1. **Get Time Information**: Calls `get_user_time_info` to get the time information.
    2. **Check for Errors**: Returns an error message if there is an error in retrieving time information.
    3. **Format Full Response**: Appends the time information to the user's query to create the full response.

In [19]:
import pytz
from datetime import datetime
from typing import Dict, Any

def get_user_time_info(user_timezone_str: str) -> Dict[str, Any]:
    """
    Get the current time and daylight saving status for the given user timezone.

    Parameters:
    - user_timezone_str (str): The user's timezone as a string.

    Returns:
    - Dict[str, Any]: A dictionary containing the formatted time information and daylight saving status.
    """
    try:
        # Get the user's time zone
        user_timezone = pytz.timezone(user_timezone_str)

        # Get the current time in the user's time zone
        current_time = datetime.now(user_timezone)

        # Check if daylight saving time is in effect
        is_daylight_saving = bool(current_time.dst())

        # Get the time zone abbreviation (e.g., PDT, PST, UTC)
        timezone_abbreviation = current_time.strftime('%Z')

        # Format the response
        time_info = {
            "user_timezone": str(user_timezone),
            "current_time": current_time.strftime('%Y-%m-%d %H:%M:%S'),
            "daylight_saving": 'in effect' if is_daylight_saving else 'not in effect',
            "timezone_abbreviation": timezone_abbreviation
        }

        return time_info

    except Exception as e:
        return {"error": f"Error retrieving time information: {e}"}

def handle_user_query(user_query: str, user_timezone_str: str) -> str:
    """
    Handle the user's query by appending the current time information.

    Parameters:
    - user_query (str): The user's query.
    - user_timezone_str (str): The user's timezone as a string.

    Returns:
    - str: The full response with the time information and user's query.
    """
    time_info = get_user_time_info(user_timezone_str)
    if "error" in time_info:
        return time_info["error"]
    
    # Add the time information after the user's query
    full_response = (
        f"{user_query} - I am in {time_info['user_timezone']}, current time is {time_info['current_time']} "
        f"{time_info['timezone_abbreviation']} and daylight saving is {time_info['daylight_saving']}."
    )
    return full_response

In [20]:
user_query = "When is the next event?"
user_timezone_str = "America/Los_Angeles"
user_query_with_time_info = handle_user_query(user_query, user_timezone_str)
print(user_query_with_time_info)

When is the next event? - I am in America/Los_Angeles, current time is 2024-08-05 22:33:40 PDT and daylight saving is in effect.


## Build JSON Response LLM

In this code, we are using the `pydantic` library to define data models and ensure data validation. We also utilize the AzureOpenAI client, enhanced by the `instructor` library, to interact with the Azure OpenAI service. The goal is to generate a JSON response that accurately represents event details based on user queries. The `instructor` library improves the accuracy and reliability of JSON formatting by integrating `pydantic` models directly into the response generation process.

#### 🔧 Initialize AzureOpenAI Client:

- The AzureOpenAI client is initialized using environment variables for the endpoint, API version, and API key.
- The `instructor` library is patched to the AzureOpenAI client to leverage the response model functionality.

#### 📝 Define Data Models:

- **`EventDetail`**: A `pydantic` model representing the details of an event. It includes fields like `eventId`, `eventDate`, `eventName`, `timeZone`, `currentTime`, and `daylightSaving`. A `model_validator` is used to ensure the event details are valid.
- **`UserQuery`**: A `pydantic` model representing the user's query details including `query`, `timeZone`, `currentTime`, and `daylightSaving`.
- **`EventResponse`**: A `pydantic` model representing the response structure. It includes a boolean `answered` and an optional `nextEvent` of type `EventDetail`.

#### ⚙️ Generate Event Response Function:

- **`generate_event_response`**: This function takes a `user_query` and a `reference_text` list as inputs.
  - It sends a request to the Azure OpenAI service using the `client.chat.completions.create` method. The `instructor` library is used to specify the `response_model` as `EventDetail` to ensure the response conforms to the expected structure.
  - The result is parsed and validated using `pydantic`. If the parsing is successful, an `EventResponse` with the event details is returned. If there is a validation error, an error message is printed, and an `EventResponse` with `answered` set to `False` is returned.

In [21]:
import instructor
from openai import AzureOpenAI
import os
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import Optional, List, Dict

# Initialize the AzureOpenAI client
client = instructor.from_openai(AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_KEY")
))

class EventDetail(BaseModel):
    event_id: int
    utc_event_date: str
    event_name: str
    user_local_time_zone: str
    user_local_event_date: str
    user_local_day_light_saving: bool

class EventResponse(BaseModel):
    answered: bool
    nextEvent: Optional[EventDetail]

def generate_event_response(user_query: str, reference_text: List[Dict[str, str]]) -> EventResponse:
    result = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": user_query
            }
        ],
        model=os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID"),
        temperature=0,
        response_model=EventDetail
    )

    # Parse the result and return as EventResponse
    if result:
        try:
            next_event_data = result.model_dump()
            next_event = EventDetail(**next_event_data)
            return EventResponse(answered=True, nextEvent=next_event)
        except ValidationError as e:
            print(f"Error in parsing event details: {e}")
            return EventResponse(answered=False, nextEvent=None)
    else:
        return EventResponse(answered=False, nextEvent=None)

In [22]:
user_query = f"{user_query_with_time_info}\n\n# REFERENCE_TEXT\n{event_dataset['Events'][0]}" 

In [23]:
generate_event_response(user_query, event_dataset['Events'][0])

EventResponse(answered=True, nextEvent=EventDetail(event_id=1001, utc_event_date='2024-08-09T00:33:22Z', event_name='Event 2', user_local_time_zone='America/Los_Angeles', user_local_event_date='2024-08-08 17:33:22', user_local_day_light_saving=True))

In [10]:
import pandas as pd
from typing import List, Dict, Any


def process_events_and_add_metadata(df: pd.DataFrame, user_query: str, user_timezone_str: str) -> pd.DataFrame:
    """
    Process each row of the DataFrame, build the prompt, run the function, and add metadata to new columns.

    Parameters:
    - df (pd.DataFrame): DataFrame containing a list of events in the "Events" column.
    - reference_text (str): The reference text to which the events will be appended.

    Returns:
    - pd.DataFrame: DataFrame with additional columns containing the metadata.
    """

    # Example usage of appending events to reference text
    user_query = handle_user_query(user_query, user_timezone_str)
    # Initialize columns for metadata
    metadata_columns = ['event_id', 'utc_event_date', 'event_name', 'user_local_time_zone', 'user_local_event_date', 'user_local_day_light_saving']
    for col in metadata_columns:
        df[col] = 'N/A'

    for index, row in df.iterrows():
        events = row['Events']
        reference_text = row['Events']
        formatted_events = "\n".join([f"- {event}" for event in events])
        full_text = f"{user_query}\n\n# REFERENCE_TEXT\n{formatted_events}"
        
        # Generate event response
        event_response = generate_event_response(full_text, reference_text)
        
        if event_response.answered:
            next_event = event_response.nextEvent
            df.at[index, 'event_id'] = next_event.event_id if next_event else 'N/A'
            df.at[index, 'utc_event_date'] = next_event.utc_event_date if next_event else 'N/A'
            df.at[index, 'event_name'] = next_event.event_name if next_event else 'N/A'
            df.at[index, 'user_local_time_zone'] = next_event.user_local_time_zone if next_event else 'N/A'
            df.at[index, 'user_local_event_date'] = next_event.user_local_event_date if next_event else 'N/A'
            df.at[index, 'user_local_day_light_saving'] = next_event.user_local_day_light_saving if next_event else 'N/A'

    return df

In [14]:
user_query = "When is the next game?"
user_timezone_str = "America/Los_Angeles"
event_dataset_test = event_dataset.copy()

processed_df = process_events_and_add_metadata(event_dataset_test, user_query, user_timezone_str)

In [13]:
processed_df

,Events,Closest_Event,Los_Angeles,Chicago,New_York,London,UTC,answered,event_id,utc_event_date,event_name,user_local_time_zone,user_local_event_date,user_local_day_light_saving
0,"[{'eventId': 1056, 'eventDate': '2024-08-27T20...","{'eventId': 1072, 'eventDate': '2024-08-13T19:...",2024-08-13 12:25:05 PDT,2024-08-13 14:25:05 CDT,2024-08-13 15:25:05 EDT,2024-08-13 20:25:05 BST,2024-08-13 19:25:05 UTC,True,1042,2024-08-16T18:59:03Z,Event 43,America/Los_Angeles,2024-08-16 11:59:03,True
1,"[{'eventId': 1080, 'eventDate': '2024-09-01T07...","{'eventId': 1061, 'eventDate': '2024-08-07T11:...",2024-08-07 04:57:21 PDT,2024-08-07 06:57:21 CDT,2024-08-07 07:57:21 EDT,2024-08-07 12:57:21 BST,2024-08-07 11:57:21 UTC,True,1061,2024-08-07T11:57:21Z,Event 62,America/Los_Angeles,2024-08-07 04:57:21,True
2,"[{'eventId': 1011, 'eventDate': '2024-08-03T15...","{'eventId': 1036, 'eventDate': '2024-08-07T16:...",2024-08-07 09:44:31 PDT,2024-08-07 11:44:31 CDT,2024-08-07 12:44:31 EDT,2024-08-07 17:44:31 BST,2024-08-07 16:44:31 UTC,True,1036,2024-08-07T16:44:31Z,Event 37,America/Los_Angeles,2024-08-07 09:44:31,True
3,"[{'eventId': 1089, 'eventDate': '2024-08-24T17...","{'eventId': 1083, 'eventDate': '2024-08-03T21:...",2024-08-03 14:01:35 PDT,2024-08-03 16:01:35 CDT,2024-08-03 17:01:35 EDT,2024-08-03 22:01:35 BST,2024-08-03 21:01:35 UTC,True,1089,2024-08-24T17:11:59Z,Event 90,America/Los_Angeles,2024-08-24 10:11:59,True
4,"[{'eventId': 1014, 'eventDate': '2024-08-07T02...","{'eventId': 1074, 'eventDate': '2024-08-06T21:...",2024-08-06 14:45:51 PDT,2024-08-06 16:45:51 CDT,2024-08-06 17:45:51 EDT,2024-08-06 22:45:51 BST,2024-08-06 21:45:51 UTC,True,1074,2024-08-06T21:45:51Z,Event 75,America/Los_Angeles,2024-08-06 14:45:51,True
5,"[{'eventId': 1006, 'eventDate': '2024-08-21T12...","{'eventId': 1048, 'eventDate': '2024-08-07T01:...",2024-08-06 18:47:09 PDT,2024-08-06 20:47:09 CDT,2024-08-06 21:47:09 EDT,2024-08-07 02:47:09 BST,2024-08-07 01:47:09 UTC,True,1048,2024-08-07T01:47:09Z,Event 49,America/Los_Angeles,2024-08-06 18:47:09,True
6,"[{'eventId': 1048, 'eventDate': '2024-08-07T01...","{'eventId': 1048, 'eventDate': '2024-08-07T01:...",2024-08-06 18:47:09 PDT,2024-08-06 20:47:09 CDT,2024-08-06 21:47:09 EDT,2024-08-07 02:47:09 BST,2024-08-07 01:47:09 UTC,True,1048,2024-08-07T01:47:09Z,Event 49,America/Los_Angeles,2024-08-06 18:47:09,True
7,"[{'eventId': 1092, 'eventDate': '2024-08-23T07...","{'eventId': 1090, 'eventDate': '2024-08-06T00:...",2024-08-05 17:11:35 PDT,2024-08-05 19:11:35 CDT,2024-08-05 20:11:35 EDT,2024-08-06 01:11:35 BST,2024-08-06 00:11:35 UTC,True,1090,2024-08-06T00:11:35Z,Event 91,America/Los_Angeles,2024-08-05 17:11:35,True
8,"[{'eventId': 1080, 'eventDate': '2024-09-01T07...","{'eventId': 1061, 'eventDate': '2024-08-07T11:...",2024-08-07 04:57:21 PDT,2024-08-07 06:57:21 CDT,2024-08-07 07:57:21 EDT,2024-08-07 12:57:21 BST,2024-08-07 11:57:21 UTC,True,1061,2024-08-07T11:57:21Z,Event 62,America/Los_Angeles,2024-08-07 04:57:21,True
9,"[{'eventId': 1089, 'eventDate': '2024-08-24T17...","{'eventId': 1019, 'eventDate': '2024-08-04T13:...",2024-08-04 06:46:34 PDT,2024-08-04 08:46:34 CDT,2024-08-04 09:46:34 EDT,2024-08-04 14:46:34 BST,2024-08-04 13:46:34 UTC,True,1078,2024-08-08T23:45:18Z,Event 79,America/Los_Angeles,2024-08-08 16:45:18,True
